### Primeira competição do Kaggle - Dataset Titanic

Objetivo: Prever corretamente as pessoas que sobreviveram.

In [1]:
import pandas as pd
import os

In [2]:
os.listdir()

['.git',
 '.ipynb_checkpoints',
 'datasets',
 'housing.ipynb',
 'Include',
 'Lib',
 'meu_melhor_modelo.pkl',
 'pyvenv.cfg',
 'Scripts',
 'share',
 'Titanic.ipynb']

In [3]:
train_data = pd.read_csv("datasets/titanic/train.csv")

In [4]:
train_data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


### Parte 1 EDA - Exploratory Data Analysis

In [5]:
train_data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [7]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [8]:
train_data.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.036847,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500
Age,0.036847,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067
SibSp,-0.057527,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000


Pclass e Fare podem ser interessantes para analise, dado que possuem certa correlação com nosso target(Survived)

In [9]:
train_data.isnull().any()

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin           True
Embarked        True
dtype: bool

Temos campos nulos o que pode prejudicar o modelo no processo de predição. 
Podemos abordar soluções distintas para esse problema. Começaremos por preencher os valores nulos.

In [10]:
def descreve_valores_nulos(dataset, nome_coluna):
    totais = dataset[nome_coluna].isnull().value_counts()
    percentual = dataset[nome_coluna].isnull().value_counts(normalize=True).mul(100).round(2).astype(str) +'%'
    return pd.DataFrame({'Totais': totais, 'Percentual': percentual})

In [11]:
descreve_valores_nulos(train_data,'Age')

,Totais,Percentual
False,714,80.13%
True,177,19.87%


In [12]:
descreve_valores_nulos(train_data,'Cabin')

,Totais,Percentual
True,687,77.1%
False,204,22.9%


Cabin torna-se forte candidato a ser removido visto que a maior parte dos seus dados estão vazios.

In [13]:
descreve_valores_nulos(train_data,'Embarked')

,Totais,Percentual
False,889,99.78%
True,2,0.22%


In [14]:
pd.set_option('display.max_rows', None)

In [15]:
train_data.Cabin.value_counts().to_frame()

,Cabin
B96 B98,4
G6,4
C23 C25 C27,4
C22 C26,3
F33,3
F2,3
E101,3
D,3
C78,2
C93,2


In [16]:
train_data.Ticket.value_counts().to_frame()

,Ticket
347082,7
CA. 2343,7
1601,7
3101295,6
CA 2144,6
347088,6
S.O.C. 14879,5
382652,5
LINE,4
PC 17757,4


Obviamente, nem todas as colunas nos serão úteis. Portanto, essas serão removidas.

In [17]:
train_data_sem_tratamento = train_data.copy()

In [18]:
train_data_sem_tratamento.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [19]:
train_data.drop("Name", axis = 1, inplace=True)
train_data.drop("Ticket", axis = 1, inplace=True)
train_data.drop("Cabin", axis = 1, inplace=True)
train_data.drop("PassengerId", axis = 1, inplace=True)

In [20]:
train_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [21]:
train_data.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [22]:
train_data.isnull().value_counts()

Survived  Pclass  Sex    Age    SibSp  Parch  Fare   Embarked
False     False   False  False  False  False  False  False       712
                         True   False  False  False  False       177
                         False  False  False  False  True          2
dtype: int64

### 2 - Tratamento dos dados

Teremos que tratar dados textuais como(Sex e Embarked) e dados númericos. Para isso utilizaremos diferentes tipos de transformadores.

In [23]:
train_data.dtypes

Survived      int64
Pclass        int64
Sex          object
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Embarked     object
dtype: object

In [24]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

#### 2.1 Criando Pipelines

In [25]:
##Pipeline para atributos númericos
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy= 'median')),
    ('std_scaler', StandardScaler())
])

A principio, para atributos númericos, trataremos os dados faltantes utilizando a mediana dos dados existentes.

In [26]:
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="most_frequent")),
    ('cat to num', OrdinalEncoder())
])

Para atributos categóricos, como no caso de Embarked, usaremos a categoria mais recorrente.

In [27]:
train_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [28]:
labels = train_data["Survived"].copy()
train_data= train_data.drop("Survived", axis=1)
atributos_numericos = list(train_data.drop(["Sex", "Embarked"], axis =1)) ##train_data.select_dtypes(exclude="object").columns 
atributos_categoricos = list(train_data[["Sex", "Embarked"]]) ##train_data.select_dtypes(include="object").columns 

In [29]:
atributos_numericos

['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [30]:
atributos_categoricos

['Sex', 'Embarked']

In [31]:
total_pipeline = ColumnTransformer([
    ("Numerico", num_pipeline, atributos_numericos),
    ("Categorico", cat_pipeline, atributos_categoricos)   
])

In [32]:
train_data_prepared = total_pipeline.fit_transform(train_data)

In [33]:
train_data_prepared

array([[ 0.82737724, -0.56573646,  0.43279337, ..., -0.50244517,
         1.        ,  2.        ],
       [-1.56610693,  0.66386103,  0.43279337, ...,  0.78684529,
         0.        ,  0.        ],
       [ 0.82737724, -0.25833709, -0.4745452 , ..., -0.48885426,
         0.        ,  2.        ],
       ...,
       [ 0.82737724, -0.1046374 ,  0.43279337, ..., -0.17626324,
         0.        ,  2.        ],
       [-1.56610693, -0.25833709, -0.4745452 , ..., -0.04438104,
         1.        ,  0.        ],
       [ 0.82737724,  0.20276197, -0.4745452 , ..., -0.49237783,
         1.        ,  1.        ]])

### Parte 3 - Treinando um modelo


In [48]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold

In [46]:
test = pd.read_csv("datasets/titanic/test.csv")

In [35]:
labels.value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [43]:
labels.value_counts().isnull().any()

False

In [50]:
lin_reg = LinearRegression()
lin_reg.fit(train_data_prepared, labels)
##train_x, test_x, train_y, test_y = train_test_split(x,y,random_state=0, test_size = 0.25, stratify = y)
resultados = cross_validate(lin_reg, x, y, cv = cv, return_train_score=False)

NameError: name 'x' is not defined

In [45]:
##regressao_linear.predict()

TypeError: predict() missing 1 required positional argument: 'X'

In [49]:
##cv = StratifiedKFold(n_splits = 3, shuffle = True)